# Import

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip3 install torch

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm
import pandas as pd
from types import SimpleNamespace

# Hyperparameter

In [24]:
config = {
    "learning_rate": 2e-5,
    "epoch": 10,
    "batch_size": 64,
}

CFG = SimpleNamespace(**config)

# Load Data

In [25]:
train_df = pd.read_csv("/content/drive/MyDrive/gbt해커톤/data/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/gbt해커톤//data/test.csv")

# Load Model

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=len(train_df['분류'].unique())).to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Custom Dataset

In [27]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# Data Preprocessing

### 1. 불용어 제거

In [28]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[A-Za-z]+[가-힣\u4E00-\u9FFF]+|'  # 영어+한글
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[A-Za-z\u4E00-\u9FFF]+|'  # 한글+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자
        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'[0-9]+|'                          # 숫자
        r'[A-Za-z]+'                        # 영어
    )
    valid_keywords = []
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            removed_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 결과 출력
print("\n수정된 데이터프레임:")
print(train_df)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


수정된 데이터프레임:
                ID        분류                                     제목  \
0      TRAIN_00000  문화:전시_공연  용인문화재단, 인문학 콘서트 ‘당신이 모르는 뮤지컬 이야기Ⅳ’ 개최   
1      TRAIN_00001        지역           용인 농촌테마파크, 7~8월 단체체험객 체험료 지원   
2      TRAIN_00002        지역        용인시, 노후주택 에너지 성능 개선 신청 18일까지 연장   
3      TRAIN_00003        지역        수원 용인 고양시,‘특례시’로 지정 도시경쟁력 증가 기대   
4      TRAIN_00004        국제      용인시, 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   
...            ...       ...                                    ...   
54604  TRAIN_54604        국제  용인 아파트서 30대 여성, 아들 딸 함께 추락 "극단 선택 추정"   
54605  TRAIN_54605  사회:교육_시험                    용인시, '위탁부모 보수교육' 실시   
54606  TRAIN_54606        지역          용인시, '플랫폼 시티' 국토부에 사업인정 협의 신청   
54607  TRAIN_54607        지역      용인시 이동읍 주민자치위원회, 저소득 20가구에 밑반찬 지원   
54608  TRAIN_54608        지역                 용인시-용인시공무원노조, 국무총리상 수상   

                                                     키워드  
0      용인문화재단, 인문학, 콘서트, 뮤지컬, 이야기, 개최, 인문학, 콘서트, 뮤지컬,...  
1      용인, 농촌, 테

In [29]:
# '분류' 열에서 앞부분만 추출하여 '분류_대분류'라는 새로운 열에 저장
train_df['분류_대분류'] = train_df['분류'].apply(lambda x: x.split(':')[0])

# 결과 확인
train_df['분류_대분류'].value_counts()

,count
분류_대분류,
지역,26950
경제,10534
사회,8245
정치,2521
문화,2500
스포츠,2035
IT_과학,1487
국제,337


In [30]:
from collections import Counter

def find_common_words_and_remove(num_categories=8, top_n=100, com_counts=10, train_df=train_df, test_df=test_df):
    # '분류_대분류'의 각 범주에 속하는 단어들을 추출
    category_words = {category: [] for category in train_df["분류_대분류"].unique()}

    for category in category_words.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드"].apply(lambda x: x.split(',')).tolist()
        category_words[category] = [word for sublist in words for word in sublist]

    # 각 범주에 속하는 단어들을 카운팅
    word_counts = {category: Counter(words) for category, words in category_words.items()}

    # 지정된 범주 갯수에 속하는 단어들을 찾기
    common_words = set()
    for word in word_counts[list(word_counts.keys())[0]].keys():
        count = sum(1 for category in word_counts.keys() if word in word_counts[category] and word_counts[category][word] >= com_counts)
        if count == num_categories:
            common_words.add(word)

    # 지정된 범주 갯수에 속하는 단어와 그 갯수를 계산
    common_word_counts = {word: sum(word_counts[category][word] for category in word_counts.keys()) for word in common_words}

    # 단어들을 갯수 기준으로 정렬하고 상위 N개를 선택
    top_common_words = sorted(common_word_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]
    print(f"상위 {top_n}개의 단어 확인")

    # 상위 N개의 단어와 그 갯수, 그리고 각 범주에서의 갯수를 출력
    for word, total_count in top_common_words:
        category_counts = {category: word_counts[category][word] for category in word_counts.keys()}
        print(f"단어: {word}, 총 갯수: {total_count}, 각 범주에서의 갯수: {category_counts}")

    # 총 몇 개의 단어가 겹치는지 출력
    print(f"총 {len(common_words)}개의 단어가 겹칩니다.")

    # top_n 기준으로 겹치는 단어들을 '키워드' 열에서 제거하고 새로운 열에 저장
    top_common_words_set = set(word for word, _ in top_common_words)

    def remove_top_common_words(keywords):
        return ', '.join([word for word in keywords.split(',') if word not in top_common_words_set])

    train_df['키워드'] = train_df['키워드'].apply(remove_top_common_words)
    test_df['키워드'] = test_df['키워드'].apply(remove_top_common_words)

In [31]:
find_common_words_and_remove(num_categories=8, com_counts=100) # top_n은 default인 100 그대로 사용
find_common_words_and_remove(num_categories=7, com_counts=300)

상위 100개의 단어 확인
단어:  용인시, 총 갯수: 90837, 각 범주에서의 갯수: {'문화': 3623, '지역': 49126, '국제': 454, '정치': 3797, '경제': 17252, '사회': 11308, '스포츠': 2574, 'IT_과학': 2703}
단어:  용인, 총 갯수: 66002, 각 범주에서의 갯수: {'문화': 3296, '지역': 33648, '국제': 255, '정치': 3687, '경제': 19063, '사회': 3755, '스포츠': 1164, 'IT_과학': 1134}
단어:  지역, 총 갯수: 52122, 각 범주에서의 갯수: {'문화': 1356, '지역': 28468, '국제': 123, '정치': 3157, '경제': 12835, '사회': 5115, '스포츠': 171, 'IT_과학': 897}
단어:  시장, 총 갯수: 48669, 각 범주에서의 갯수: {'문화': 1619, '지역': 25448, '국제': 322, '정치': 4017, '경제': 13272, '사회': 2239, '스포츠': 798, 'IT_과학': 954}
단어:  경기도, 총 갯수: 43090, 각 범주에서의 갯수: {'문화': 1244, '지역': 24280, '국제': 124, '정치': 2241, '경제': 9083, '사회': 3823, '스포츠': 1743, 'IT_과학': 552}
단어:  경기, 총 갯수: 38839, 각 범주에서의 갯수: {'문화': 1052, '지역': 14922, '국제': 225, '정치': 2246, '경제': 9233, '사회': 7061, '스포츠': 3502, 'IT_과학': 598}
단어:  이날, 총 갯수: 17324, 각 범주에서의 갯수: {'문화': 744, '지역': 7712, '국제': 128, '정치': 1892, '경제': 2320, '사회': 3817, '스포츠': 506, 'IT_과학': 205}
단어:  상황, 총 갯수: 12505, 각 범주에서의 갯수: {'문화': 38

In [32]:
from collections import Counter

def count_and_remove_low_occurrence_keywords(train_df, test_df, threshold=3):
    # '키워드' 열의 각 값을 쉼표로 분리하여 리스트로 변환
    train_df['키워드_리스트'] = train_df['키워드'].apply(lambda x: x.split(','))
    test_df['키워드_리스트'] = test_df['키워드'].apply(lambda x: x.split(','))

    # '분류_대분류' 별로 단어들을 추출하고 카운팅
    category_keywords = {category: [] for category in train_df["분류_대분류"].unique()}

    for category in category_keywords.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드_리스트"].tolist()
        category_keywords[category] = [word.strip() for sublist in words for word in sublist]

    # 각 '분류_대분류' 별로 단어들을 카운팅하고 단어가 threshold 이하로 존재하는 경우를 찾기
    low_occurrence_words = set()
    for category, words in category_keywords.items():
        word_counts = Counter(words)
        low_occurrence_words.update({word for word, count in word_counts.items() if count <= threshold})

    # 단어가 threshold 이하로 존재하는 경우를 '키워드' 열에서 제거
    def remove_low_occurrence_words(keywords):
        return ', '.join([word.strip() for word in keywords.split(',') if word.strip() not in low_occurrence_words])

    train_df['키워드'] = train_df['키워드'].apply(remove_low_occurrence_words)

    # test_df에서도 동일한 단어를 제거
    def remove_low_occurrence_words_from_test(keywords):
        return ', '.join([word.strip() for word in keywords.split(',') if word.strip() not in low_occurrence_words])

    test_df['키워드'] = test_df['키워드'].apply(remove_low_occurrence_words_from_test)

    # 제거 후 각 '분류_대분류' 별로 하위 10개의 단어와 그 갯수를 출력
    for category in category_keywords.keys():
        words = train_df.loc[train_df["분류_대분류"] == category, "키워드"].apply(lambda x: x.split(',')).tolist()
        words = [word.strip() for sublist in words for word in sublist]
        word_counts = Counter(words)
        bottom_keywords = word_counts.most_common()[:-11:-1]
        print(f"분류_대분류: {category}")
        for word, count in bottom_keywords:
            print(f"  단어: {word}, 갯수: {count}")
        print()

In [33]:
# 갯수 2개 이하인 단어들을 제거
count_and_remove_low_occurrence_keywords(train_df, test_df, threshold=2)

분류_대분류: 문화
  단어: , 갯수: 1
  단어: 대상어, 갯수: 3
  단어: 우서일절, 갯수: 3
  단어: 오수경, 갯수: 3
  단어: 칠순잔치, 갯수: 3
  단어: 서병덕, 갯수: 3
  단어: 김옹, 갯수: 3
  단어: 위작, 갯수: 3
  단어: 용자봉, 갯수: 3
  단어: 곽승진, 갯수: 3

분류_대분류: 지역
  단어: , 갯수: 1
  단어: 온세미, 갯수: 3
  단어: 모멘트, 갯수: 3
  단어: 통합데이터센터, 갯수: 3
  단어: 통합데이터, 갯수: 3
  단어: 임의장, 갯수: 3
  단어: 강서대, 갯수: 3
  단어: 닭칼국수, 갯수: 3
  단어: 요금군, 갯수: 3
  단어: 나군, 갯수: 3

분류_대분류: 국제
  단어: 낭비, 갯수: 3
  단어: 조선인, 갯수: 3
  단어: 트럼프행정부, 갯수: 3
  단어: 아시아인들, 갯수: 3
  단어: 주씨, 갯수: 3
  단어: 호감도, 갯수: 3
  단어: 따오기, 갯수: 3
  단어: 윌리엄슨카운티, 갯수: 3
  단어: 패어펙스카운티, 갯수: 3
  단어: 오세올라카운티, 갯수: 3

분류_대분류: 정치
  단어: 법률비서관, 갯수: 3
  단어: 총괄부위원장, 갯수: 3
  단어: 열병식, 갯수: 3
  단어: 전봉학, 갯수: 3
  단어: 지방지, 갯수: 3
  단어: 수원갈비, 갯수: 3
  단어: 일탈행동, 갯수: 3
  단어: 독감, 갯수: 3
  단어: 한국여성유권자경기연맹, 갯수: 3
  단어: 베네피트, 갯수: 3

분류_대분류: 경제
  단어: 강만희, 갯수: 3
  단어: 갓슬라, 갯수: 3
  단어: 배민라이더스, 갯수: 3
  단어: 파워콜, 갯수: 3
  단어: 겨울옷, 갯수: 3
  단어: 점박이물범, 갯수: 3
  단어: 개선율, 갯수: 3
  단어: 철산자이더헤리티지, 갯수: 3
  단어: 고립은둔청년, 갯수: 3
  단어: 용인마평, 갯수: 3

분류_대분류: 사회
  단어: , 갯수: 2
  단어: 말벌, 갯수: 3
  단어

In [34]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54609 entries, 0 to 54608
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ID       54609 non-null  object
 1   분류       54609 non-null  object
 2   제목       54609 non-null  object
 3   키워드      54609 non-null  object
 4   분류_대분류   54609 non-null  object
 5   키워드_리스트  54609 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [35]:
train_df.drop(columns=['제목', '분류_대분류', '키워드_리스트'], inplace=True)
test_df.drop(columns=['제목', '키워드_리스트'], inplace=True)

In [36]:
# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(train_df['분류'].unique())}
train_df['label'] = train_df['분류'].map(label_encoder)

# 데이터 분할 (train -> train + validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['분류'], random_state=42)

# 데이터셋 생성
train_dataset = TextDataset(train_df.키워드.tolist(), train_df.label.tolist(), tokenizer)
val_dataset = TextDataset(val_df.키워드.tolist(), val_df.label.tolist(), tokenizer)
test_dataset = TextDataset(test_df.키워드.tolist(), None, tokenizer)  # 라벨 없음

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False)

In [37]:
# 옵티마이저 및 학습 파라미터 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [38]:
# 학습
model.train()
best_f1 = 0.0
patience = 2  # 성능 향상이 없을 때 기다리는 에포크 수
patience_counter = 0

for epoch in range(CFG.epoch):
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epoch}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_predictions = []
    val_true_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())

    # 검증 결과 출력
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    print(f"Validation F1 Score: {val_f1:.2f}")

    # 조기 종료 체크
    if val_f1 > best_f1:
        best_f1 = val_f1
        patience_counter = 0  # 성능 향상이 있었으므로 카운터 초기화
        # 모델 저장 등 추가 작업을 여기서 수행할 수 있습니다.
    else:
        patience_counter += 1

    # patience 초과 시 학습 종료
    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Validating: 100%|██████████| 171/171 [01:27<00:00,  1.95it/s]


Validation F1 Score: 0.19


Validating: 100%|██████████| 171/171 [01:27<00:00,  1.96it/s]


Validation F1 Score: 0.29


Validating: 100%|██████████| 171/171 [01:27<00:00,  1.95it/s]


...

Validation F1 Score: 0.49


Validating: 100%|██████████| 171/171 [01:25<00:00,  1.99it/s]


Validation F1 Score: 0.50


Validating: 100%|██████████| 171/171 [01:26<00:00,  1.98it/s]

Validation F1 Score: 0.51


# Inference

In [39]:
# 테스트 세트 추론
model.eval()
test_predictions = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
        test_predictions.extend(preds.cpu().tolist())

# 라벨 디코딩
label_decoder = {i: label for label, i in label_encoder.items()}
decoded_predictions = [label_decoder[pred] for pred in test_predictions]

Testing: 100%|██████████| 366/366 [03:10<00:00,  1.92it/s]


# Submission

In [40]:
sample_submission = pd.read_csv("/content/drive/MyDrive/gbt해커톤/data/sample_submission.csv")
sample_submission["분류"] = decoded_predictions

sample_submission.to_csv("/content/drive/MyDrive/gbt해커톤/submission/1001_submission.csv", encoding='UTF-8-sig', index=False)

In [41]:
sample_submission['분류'].value_counts()

,count
분류,
지역,12551
경제:부동산,1419
사회:사건_사고,1098
경제:반도체,991
사회:사회일반,612
정치:국회_정당,474
사회:교육_시험,383
사회:의료_건강,370
스포츠:올림픽_아시안게임,357
